Pretrained Models

Pretrained models in deep learning are trained on large datasets. Here are some popular pretrained models:

1. **VGG (Visual Geometry Group) Models:**
   - VGG16
   - VGG19

2. **ResNet (Residual Networks) Models:**
   - ResNet-18
   - ResNet-34
   - ResNet-50
   - ResNet-101
   - ResNet-152

3. **Inception Models:**
   - InceptionV3
   - InceptionResNetV2

4. **MobileNet Models:**
   - MobileNetV1
   - MobileNetV2
   - MobileNetV3

5. **DenseNet Models:**
   - DenseNet-121
   - DenseNet-169
   - DenseNet-201

6. **EfficientNet Models:**
   - EfficientNetB0, B1, B2, B3, B4, B5, B6, B7

7. **Xception (Extreme Inception) Model**

8. **ResNeXt Models:**
   - ResNeXt-50
   - ResNeXt-101

9. **SqueezeNet Model**

10. **NASNet (Neural Architecture Search Network) Models:**
    - NASNetLarge
    - NASNetMobile

11. **SENet (Squeeze-and-Excitation Networks) Models:**
    - SENet-154
    - SENet-50

12. **ShuffleNet Models:**
    - ShuffleNetV1
    - ShuffleNetV2



Choosen Models: "a5tarthom according to my preferences"

VGG19
ResNet-50
InceptionV3
MobileNetV3 "MobileNetV3 is not directly available in torchvision"
DenseNet-169

VGG19

In [31]:
# Import necessary PyTorch modules
import torch
import torch.nn as nn
import torchvision.models as models


# Define a class for VGG19 without the last layer
class VGG19WithoutLastLayer(nn.Module):
    def __init__(self, num_classes=1000):
        super(VGG19WithoutLastLayer, self).__init__()

        # Load the pretrained VGG19 model from torchvision
        vgg19 = models.vgg19(weights="DEFAULT")
        print(nn.Sequential(*list(vgg19.children())[:-1]))
        # Extract the feature layers (convolutional and pooling layers)
        self.features = vgg19.features
        # print(vgg19)
        # Global average pooling layer
        self.avgpool = vgg19.avgpool
        # print(self.avgpool)
        # Extract the classifier layers, excluding the last fully connected layer
        self.classifier = nn.Sequential(*list(vgg19.classifier.children())[:-1])
        # print(vgg19.classifier)

    def forward(self, x):
        # Forward pass through the feature layers
        x = self.features(x)

        # Global average pooling
        x = self.avgpool(x)

        # Flatten the output
        x = x.view(x.size(0), -1)

        # Forward pass through the classifier layers
        x = self.classifier(x)
        return x


# Function to add a custom last layer to the base model
def add_custom_last_layer(base_model, num_classes):
    # Identify the index of the last linear layer in the classifier
    last_layer_index = None
    for i, layer in enumerate(reversed(base_model.classifier)):
        if isinstance(layer, nn.Linear):
            last_layer_index = len(base_model.classifier) - 1 - i
            break

    # Remove the dropout layers if present
    if last_layer_index is not None:
        base_model.classifier = base_model.classifier[:last_layer_index + 1]

    # Add a linear layer (fully connected) with the specified number of classes
    last_layer = nn.Linear(
        in_features=base_model.classifier[-1].out_features, out_features=num_classes
    )

    # Create a new model by appending the custom last layer to the base model
    model = nn.Sequential(base_model, last_layer)
    return model



# Function to train the model
def train_model(
    model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device="cuda"
):
    # Move the model to the specified device (e.g., GPU)
    model.to(device)

    # Iterate over epochs
    for epoch in range(num_epochs):
        # Set the model to training mode
        model.train()

        # Iterate over the training dataset
        for inputs, labels in train_loader:
            # Move inputs and labels to the device
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Calculate the loss
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        # Validation
        # Set the model to evaluation mode
        model.eval()

        # Initialize variables for calculating accuracy and loss
        val_loss = 0.0
        correct = 0
        total = 0

        # Disable gradient computation during validation
        with torch.no_grad():
            # Iterate over the validation dataset
            for inputs, labels in val_loader:
                # Move inputs and labels to the device
                inputs, labels = inputs.to(device), labels.to(device)

                # Forward pass
                outputs = model(inputs)

                # Calculate the loss
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Calculate accuracy
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # Print epoch-wise statistics
        print(
            f"Epoch {epoch + 1}/{num_epochs}, Loss: {val_loss/len(val_loader)}, Accuracy: {100 * correct / total}%"
        )


# Example usage: BUT IN OUR CASE WE WILL USE METHOD WHICH IS IN THE TOOL
# (Assuming you have train_loader and val_loader for your dataset)

# Create VGG19 base model without the last layer
base_model = VGG19WithoutLastLayer()

# Add a custom last layer based on your task
num_classes = 10  # Modify based on your classification task
model = add_custom_last_layer(base_model, num_classes)

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
# train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

ResNet-50

In [18]:
import torch
import torch.nn as nn
import torchvision.models as models

class ResNet50WithoutLastLayer(nn.Module):
    def __init__(self, num_classes=1000):
        super(ResNet50WithoutLastLayer, self).__init__()

        # Load the pretrained ResNet-50 model from torchvision
        resnet50 = models.resnet50(weights="DEFAULT")

        # Extract all layers except the last fully connected layer
        self.features = nn.Sequential(*list(resnet50.children())[:-1])

    def forward(self, x):
        # Forward pass through the feature layers
        x = self.features(x)

        # Global average pooling
        x = x.view(x.size(0), -1)

        return x


def add_custom_last_layer(base_model, num_classes):
    # Find the last AdaptiveAvgPool2d layer
    last_pooling_layer = None
    for layer in reversed(base_model.features):
        if isinstance(layer, nn.AdaptiveAvgPool2d):
            last_pooling_layer = layer
            break

    if last_pooling_layer is not None:
        # Determine the number of output features from the last pooling layer
        num_features = last_pooling_layer.output_size[0] * last_pooling_layer.output_size[1]
    else:
        # If AdaptiveAvgPool2d layer is not found, use the number of in_features of the last layer in base_model.features
        num_features = base_model.features[-1][0].in_features

    # Add a linear layer (fully connected) with the specified number of classes
    last_layer = nn.Linear(in_features=num_features, out_features=num_classes)

    # Create a new model by appending the custom last layer to the base model
    model = nn.Sequential(base_model, last_layer)
    return model


def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device="cuda"):
    # Move the model to the specified device (e.g., GPU)
    model.to(device)

    # Iterate over epochs
    for epoch in range(num_epochs):
        # Set the model to training mode
        model.train()

        # Iterate over the training dataset
        for inputs, labels in train_loader:
            # Move inputs and labels to the device
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Calculate the loss
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        # Validation
        # Set the model to evaluation mode
        model.eval()

        # Initialize variables for calculating accuracy and loss
        val_loss = 0.0
        correct = 0
        total = 0

        # Disable gradient computation during validation
        with torch.no_grad():
            # Iterate over the validation dataset
            for inputs, labels in val_loader:
                # Move inputs and labels to the device
                inputs, labels = inputs.to(device), labels.to(device)

                # Forward pass
                outputs = model(inputs)

                # Calculate the loss
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Calculate accuracy
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # Print epoch-wise statistics
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {val_loss/len(val_loader)}, Accuracy: {100 * correct / total}%")

# Example usage:
# (Assuming you have train_loader and val_loader for your dataset)

# Create ResNet-50 base model without the last layer
base_model = ResNet50WithoutLastLayer()

# Add a custom last layer based on your task
num_classes = 10  # Modify based on your classification task
model = add_custom_last_layer(base_model, num_classes)

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
# train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)


InceptionV3

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models


class InceptionV3WithoutLastLayer(nn.Module):
    def __init__(self, num_classes=1000):
        super(InceptionV3WithoutLastLayer, self).__init__()

        # Load the pretrained InceptionV3 model from torchvision
        inception_v3 = models.inception_v3(pretrained=True, aux_logits=False)

        # Extract all layers except the last fully connected layer
        self.features = nn.Sequential(*list(inception_v3.children())[:-1])

    def forward(self, x):
        # Forward pass through the feature layers
        x = self.features(x)

        # Global average pooling
        x = x.view(x.size(0), -1)

        return x


def add_custom_last_layer(base_model, num_classes):
    # Add your custom last layer based on the task (e.g., classification)
    last_layer = nn.Linear(
        in_features=base_model.features[-1][-1].in_features, out_features=num_classes
    )

    # Create a new model by appending the custom last layer to the base model
    model = nn.Sequential(base_model, last_layer)
    return model


def train_model(
    model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device="cuda"
):
    # Move the model to the specified device (e.g., GPU)
    model.to(device)

    # Iterate over epochs
    for epoch in range(num_epochs):
        # Set the model to training mode
        model.train()

        # Iterate over the training dataset
        for inputs, labels in train_loader:
            # Move inputs and labels to the device
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Calculate the loss
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        # Validation
        # Set the model to evaluation mode
        model.eval()

        # Initialize variables for calculating accuracy and loss
        val_loss = 0.0
        correct = 0
        total = 0

        # Disable gradient computation during validation
        with torch.no_grad():
            # Iterate over the validation dataset
            for inputs, labels in val_loader:
                # Move inputs and labels to the device
                inputs, labels = inputs.to(device), labels.to(device)

                # Forward pass
                outputs = model(inputs)

                # Calculate the loss
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Calculate accuracy
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # Print epoch-wise statistics
        print(
            f"Epoch {epoch + 1}/{num_epochs}, Loss: {val_loss/len(val_loader)}, Accuracy: {100 * correct / total}%"
        )


# Example usage:
# (Assuming you have train_loader and val_loader for your dataset)

# Create InceptionV3 base model without the last layer
base_model = InceptionV3WithoutLastLayer()

# Add a custom last layer based on your task
num_classes = 10  # Modify based on your classification task
model = add_custom_last_layer(base_model, num_classes)

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
# train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

DenseNet-169

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models


class DenseNet169WithoutLastLayer(nn.Module):
    def __init__(self, num_classes=1000):
        super(DenseNet169WithoutLastLayer, self).__init__()

        # Load the pretrained DenseNet-169 model from torchvision
        densenet169 = models.densenet169(pretrained=True)

        # Extract all layers except the last fully connected layer
        self.features = densenet169.features

    def forward(self, x):
        # Forward pass through the feature layers
        x = self.features(x)

        # Global average pooling
        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = x.view(x.size(0), -1)

        return x


def add_custom_last_layer(base_model, num_classes):
    # Add your custom last layer based on the task (e.g., classification)
    last_layer = nn.Linear(
        in_features=base_model.features[-1].in_features, out_features=num_classes
    )

    # Create a new model by appending the custom last layer to the base model
    model = nn.Sequential(base_model, last_layer)
    return model


def train_model(
    model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device="cuda"
):
    # Move the model to the specified device (e.g., GPU)
    model.to(device)

    # Iterate over epochs
    for epoch in range(num_epochs):
        # Set the model to training mode
        model.train()

        # Iterate over the training dataset
        for inputs, labels in train_loader:
            # Move inputs and labels to the device
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Calculate the loss
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        # Validation
        # Set the model to evaluation mode
        model.eval()

        # Initialize variables for calculating accuracy and loss
        val_loss = 0.0
        correct = 0
        total = 0

        # Disable gradient computation during validation
        with torch.no_grad():
            # Iterate over the validation dataset
            for inputs, labels in val_loader:
                # Move inputs and labels to the device
                inputs, labels = inputs.to(device), labels.to(device)

                # Forward pass
                outputs = model(inputs)

                # Calculate the loss
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Calculate accuracy
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # Print epoch-wise statistics
        print(
            f"Epoch {epoch + 1}/{num_epochs}, Loss: {val_loss/len(val_loader)}, Accuracy: {100 * correct / total}%"
        )


# Example usage:
# (Assuming you have train_loader and val_loader for your dataset)

# Create DenseNet-169 base model without the last layer
base_model = DenseNet169WithoutLastLayer()

# Add a custom last layer based on your task
num_classes = 10  # Modify based on your classification task
model = add_custom_last_layer(base_model, num_classes)

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
# train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

LenNet "Omar's Recommendation"

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models


class LeNetWithoutLastLayer(nn.Module):
    def __init__(self):
        super(LeNetWithoutLastLayer, self).__init__()
        self.lenet=models.googlenet(weights="DEFAULT")
        # LeNet architecture: conv1 -> relu -> pool1 -> conv2 -> relu -> pool2
        self.features=  nn.Sequential(*list(self.lenet.children())[:-1])
        print(self.features)
        # = nn.Sequential(
        #     nn.Conv2d(1, 6, kernel_size=5),
        #     nn.ReLU(),
        #     nn.MaxPool2d(kernel_size=2, stride=2),
        #     nn.Conv2d(6, 16, kernel_size=5),
        #     nn.ReLU(),
        #     nn.MaxPool2d(kernel_size=2, stride=2),
        # )

    def forward(self, x):
        x = self.features(x)
        return x


def add_custom_last_layer(base_model, num_classes):
    # Access the output channels of the last layer (Inception module)
    last_module = base_model.features[-1][-1]
    
    # Check if the last module is an inception module
    if isinstance(last_module, nn.modules.container.ModuleList):
        last_layer_out_channels = last_module[-1].out_channels
    else:
        last_layer_out_channels = last_module.out_channels

    # Add your custom last layer based on the task (e.g., classification)
    last_layer = nn.Linear(
        in_features=last_layer_out_channels, out_features=num_classes
    )

    # Create a new model by appending the custom last layer to the base model
    model = nn.Sequential(base_model, last_layer)
    return model




def train_model(
    model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device="cuda"
):
    # Move the model to the specified device (e.g., GPU)
    model.to(device)

    # Iterate over epochs
    for epoch in range(num_epochs):
        # Set the model to training mode
        model.train()

        # Iterate over the training dataset
        for inputs, labels in train_loader:
            # Move inputs and labels to the device
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Calculate the loss
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        # Validation
        # Set the model to evaluation mode
        model.eval()

        # Initialize variables for calculating accuracy and loss
        val_loss = 0.0
        correct = 0
        total = 0

        # Disable gradient computation during validation
        with torch.no_grad():
            # Iterate over the validation dataset
            for inputs, labels in val_loader:
                # Move inputs and labels to the device
                inputs, labels = inputs.to(device), labels.to(device)

                # Forward pass
                outputs = model(inputs)

                # Calculate the loss
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Calculate accuracy
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # Print epoch-wise statistics
        print(
            f"Epoch {epoch + 1}/{num_epochs}, Loss: {val_loss/len(val_loader)}, Accuracy: {100 * correct / total}%"
        )


# Example usage:
# (Assuming you have train_loader and val_loader for your dataset)

# Create LeNet base model without the last layer
base_model = LeNetWithoutLastLayer()

# Add a custom last layer based on your task
num_classes = 10  # Modify based on your classification task
model = add_custom_last_layer(base_model, num_classes)

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Train the model
# train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

Sequential(
  (0): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (4): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (5): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (bra

TypeError: 'Dropout' object is not subscriptable